In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Filtered_Global_Terror_Data.csv',index_col='Unnamed: 0')

In [79]:
df.columns.to_list() # asıl verisetinden çektiğim kolonlar

['iyear',
 'imonth',
 'iday',
 'country_txt',
 'city',
 'COUNTRY_LATITUDE',
 'COUNTRY_LONGITUDE',
 'CITY_LATITUDE',
 'CITY_LONGITUDE',
 'success',
 'nkill',
 'nwound',
 'nreleased',
 'location',
 'summary',
 'addnotes',
 'claimmode_txt',
 'attacktype1_txt',
 'weapdetail',
 'target1',
 'targtype1_txt',
 'corp1',
 'date']

In [3]:
# df['date'] = df.apply(lambda x: f"{x['iyear']}-{x['iday']}-{x['imonth']}",axis=1) # ayrı ayrı olan 3 kolonu birleştirdik

In [ ]:
"""
# tarih formatını düzenlemek için kendimize bir fonksiyon yazıyoruz
def tarih_birlestir(cols):
    gun = cols['iday']
    ay = cols['imonth']
    yil = cols['iyear']
    if int(gun) < 10:
        if int(gun) == 0:
            gun = 1
        gun = f"0{gun}"
    if int(ay) < 10:
        if int(ay) == 0:
            ay = 1
        ay = f"0{ay}"
    tarih = f"{gun}-{ay}-{yil}"
    return tarih
"""

In [ ]:
#df['date'] = df[['iday','imonth','iyear']].apply(tarih_birlestir,axis=1)

In [ ]:
#pd.to_datetime(df['date'],format='%d-%m-%Y') # tarih formatına göre date kolonu ekliyoruz

In [14]:
# Bu satır bir kere çalışmıştır. Boş veriler dolduruldu ve daha sonra csv olarak dosya dışarı aktarıldı. Bir daha bu satır çalışmadı
"""
#df['nreleased'].replace(-1,np.nan,inplace=True)
#df['nkill'].replace(-1,np.nan,inplace=True)
#df['nwound'].replace(-1,np.nan,inplace=True)
#df['success'].replace(-1,np.nan,inplace=True)
df['location'].replace(np.nan,'Bilinmiyor',inplace=True)
df['summary'].replace(np.nan,'Bilinmiyor',inplace=True)
df['addnotes'].replace(np.nan,'Bilinmiyor',inplace=True)
df['claimmode_txt'].replace(np.nan,'Bilinmiyor',inplace=True)
df['attacktype1_txt'].replace(np.nan,'Bilinmiyor',inplace=True)
df['weapdetail'].replace(np.nan,'Bilinmiyor',inplace=True)
df['target1'].replace(np.nan,'Bilinmiyor',inplace=True)
df['targtype1_txt'].replace(np.nan,'Bilinmiyor',inplace=True)
df['corp1'].replace(np.nan,'Bilinmiyor',inplace=True)
df.to_csv('Filtered_Global_Terror_Data.csv')
"""

In [4]:
df['date'] = pd.to_datetime(df['date']) # date kolonunu datetime tipine çeviriyoruz

In [5]:
import plotly.express as px
# mapbox styles -> mapbox tokenimiz yoksa update_layout ile bu stilleri set etmemiz gerek
#open-street-map
#stamen-terrain

In [6]:
# ÜLKE ENLEM, BOYLAM VE ADINA GÖRE FİLTRELİYORUZ. Mantıken 3ü'de aynı olan veriler aynı yerde gerçekleşmiştir
# as_index=False gruplanan verilerin indexlenmesini istemiyoruz
earth_df = df.groupby(by=['COUNTRY_LATITUDE','COUNTRY_LONGITUDE','country_txt'],as_index=False)

In [7]:
fig = px.scatter_geo(earth_df.sum(), color="country_txt",
                     hover_name="country_txt", size="nkill",
                     lat="COUNTRY_LATITUDE", lon="COUNTRY_LONGITUDE",
                     projection="natural earth",labels={'country_txt':'Ülke Adı',
                                                        'nkill':'Toplam Ölen',
                                                        'COUNTRY_LATITUDE':'Ülke Enlem',
                                                       'COUNTRY_LONGITUDE':'Ülke Boylam'},
                    )

fig.update_layout(title='Dünya Çapında Terör Saldırıları',
                 )
fig.write_html('earth-map.html')

In [8]:
# GRUPLANDIRIRKEN YILLARI DA DAHİL EDİYORUZ. BU SAYEDE ANIMATED'I YILLARA GÖRE YAPABİLECEĞİZ
animated_earth_df = df.groupby(by=['COUNTRY_LATITUDE','COUNTRY_LONGITUDE','country_txt','iyear'],as_index=False)

In [9]:
fig = px.scatter_geo(animated_earth_df.sum(), color="country_txt",
                     hover_name="country_txt", size="nkill",
                     lat="COUNTRY_LATITUDE", lon="COUNTRY_LONGITUDE",
                     projection="natural earth",labels={'country_txt':'Ülke Adı',
                                                        'nkill':'Toplam Ölen',
                                                        'COUNTRY_LATITUDE':'Ülke Enlem',
                                                       'COUNTRY_LONGITUDE':'Ülke Boylam',
                                                       'iyear':'Yıl'},
                     animation_frame="iyear"
                    )

fig.update_layout(title='Yıl YIl Dünya Çapında Terör Saldırıları',
                 )

fig.write_html('animated-earth-map.html')

In [10]:
# ülke filtrelenince ülkeye göre bu harita çalışacak
scatter_df = df.query(f"country_txt == 'Turkey'")
scatter_df['nkill'].replace(np.nan,0.000001,inplace=True)

/home/baysan/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
# filtreli bir durumda ülkelere özel olarak bu haritayı kullanacağız


fig = px.scatter_mapbox(scatter_df, lat="CITY_LATITUDE", lon="CITY_LONGITUDE",     
                        color="city", size='nkill',
                        hover_data=['summary'],
                  color_continuous_scale=px.colors.cyclical.IceFire, zoom=5)
fig.update_layout(mapbox_style='open-street-map')
fig.write_html('scatter_map.html')

In [15]:
df['date']

0        1996-03-28
1        1974-12-15
2        1998-07-03
3        2014-05-28
4        1987-10-04
            ...    
170303   2013-11-23
170304   2013-11-23
170305   2013-11-23
170306   2013-11-23
170307   2013-11-23
Name: date, Length: 170308, dtype: datetime64[ns]

In [16]:
df.columns

Index(['iyear', 'imonth', 'iday', 'country_txt', 'city', 'COUNTRY_LATITUDE',
       'COUNTRY_LONGITUDE', 'CITY_LATITUDE', 'CITY_LONGITUDE', 'success',
       'nkill', 'nwound', 'nreleased', 'location', 'summary', 'addnotes',
       'claimmode_txt', 'attacktype1_txt', 'weapdetail', 'target1',
       'targtype1_txt', 'corp1', 'date'],
      dtype='object')

In [32]:
df['weapdetail']

0                                                Patlayıcı
1                                                Patlayıcı
2                                               Bilinmiyor
3                                               Bilinmiyor
4                                    Otomatik ateşli silah
                                ...                       
170303                                          Bilinmiyor
170304      Saldırıda bilinmeyen bir patlayıcı kullanıldı.
170305                                          Bilinmiyor
170306             Saldırıda bir araba bombası kullanıldı.
170307    Saldırıda bilinmeyen ateşli silahlar kullanıldı.
Name: weapdetail, Length: 170308, dtype: object

In [62]:
grouped = df.groupby(['weapdetail'],as_index=False).sum().sort_values('nkill',ascending=False) # nkill'e göre sıralıyoruz

In [64]:
grouped.set_index('weapdetail',inplace=True)# index'i weapdetail kolonu olarak değiştiriyoruz

In [65]:
grouped.drop(index='Bilinmiyor',inplace=True) # Bilimiyor olan index'i kaldırıyoruz

In [66]:
grouped.reset_index(inplace=True) # index'leri sıfırlıyoruz

In [67]:
grouped.iloc[0:10,:] # en yüksek nkill'i olan 10 satırı alıyoruz

,weapdetail,iyear,imonth,iday,COUNTRY_LATITUDE,COUNTRY_LONGITUDE,CITY_LATITUDE,CITY_LONGITUDE,success,nkill,nwound,nreleased
0,Otomatik ateşli silah,26801505,86921,205724,164261.855669,-381828.956907,163811.847880,-298047.413293,13093.0,58391.0,17558.0,311.0
1,Patlayıcı,42576613,137916,331739,344170.071548,-423569.068556,346545.687790,-344278.636007,19491.0,12145.0,42168.0,310.0
2,ateşli silâh,8293902,27709,65779,117524.244650,36555.112739,116434.988928,32818.531706,3886.0,9255.0,3465.0,113.0
3,Saldırıda bilinmeyen ateşli silahlar kullanıldı.,7809852,25980,61055,96833.396301,245448.990711,93559.018942,230818.135830,3689.0,6318.0,3561.0,-10292.0
4,Tabanca,9193524,28669,69986,78635.402552,-28110.267653,82105.516140,-30606.457565,4142.0,4853.0,1700.0,935.0
5,Saldırıda doğaçlama patlayıcı bir cihaz kullan...,5922581,19803,46776,91255.590161,155443.943004,90103.827227,152386.263794,2609.0,3714.0,11959.0,-393.0
6,Bıçak,1474329,5178,11799,19156.677458,17498.929268,17882.245853,15680.116309,604.0,3573.0,879.0,544.0
7,Saldırıda bir yol kenarı bombası kullanıldı.,5024486,15584,38683,78934.829100,131852.514268,77743.290811,127217.964274,2319.0,3101.0,6706.0,0.0
8,Saldırıda bilinmeyen silahlar kullanıldı.,1466989,4741,10840,16408.379589,43014.788948,15643.213130,40611.950194,693.0,2188.0,1137.0,-8007.0
9,Araba bombası,891228,3015,7506,13593.105472,494.746303,13457.022139,-134.789931,385.0,1504.0,5956.0,0.0


In [100]:
grouped = df.groupby(['attacktype1_txt','country_txt'],as_index=False).sum().sort_values('nkill',ascending=False) # nkill'e göre sıralıyoruz

In [122]:
grouped = df.groupby(by=['iyear','attacktype1_txt'], as_index=False).sum()

In [126]:
grouped

,iyear,attacktype1_txt,imonth,iday,COUNTRY_LATITUDE,COUNTRY_LONGITUDE,CITY_LATITUDE,CITY_LONGITUDE,success,nkill,nwound,nreleased
0,1970,Bilinmeyen,21,87,36.189351,388.766790,65.595856,366.549339,4.0,4.0,2.0,0.0
1,1970,Bilinmiyor,4,16,73.370026,-183.690155,81.463769,-147.959330,0.0,0.0,0.0,0.0
2,1970,Bombalama / Patlama,2006,4918,11416.074449,-24347.230855,11691.888557,-24023.572936,266.0,96.0,124.0,0.0
3,1970,Rehin Alma (Kaçırma),243,570,94.949156,-1757.952344,125.148902,-1473.954236,33.0,6.0,1.0,40.0
4,1970,Rehine Alma (Barikat Olayı),20,39,28.744742,15.789491,29.008687,15.677974,3.0,0.0,0.0,120.0
...,...,...,...,...,...,...,...,...,...,...,...,...
445,2016,Silahlı saldırı,9470,23037,36952.807651,83239.665165,37273.780947,81227.373255,1310.0,3741.0,2693.0,0.0
446,2016,Silahsız Assault,266,581,1204.661682,1915.085371,1263.410686,1864.887831,38.0,5.0,405.0,0.0
447,2016,Suikast,4064,9610,11158.203940,41099.931325,11357.403193,37586.191404,395.0,574.0,448.0,5.0
448,2016,Tesis / Altyapı Saldırısı,2619,5964,10670.111295,15601.204650,10958.116164,14784.274652,364.0,92.0,3.0,-99.0


In [249]:
test = df[['iyear','attacktype1_txt']].copy() # asıl df'in belirli bir kesitinin kopyasını değişkene atıyoruz

In [250]:
test['count'] = 1 # yeni kolon ekliyoruz

In [263]:
grpd = test.groupby(['iyear','attacktype1_txt'],as_index=False).sum() # grupluyoruz

In [275]:
grpd.sort_values(by='count',ascending=False,inplace=True) # count değişkenine göre sıralıyoruz